# *Praca domowa 2*
Mikołaj Roguski


# import danch oraz przygotowanie ich do klasyfikacji binarnej

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

### import danych

In [2]:
df = pd.read_csv('dfout.csv')

### przypomnienie jak wyglądają dane

In [3]:
df.shape

(98044, 78)

In [6]:
df.describe()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,...,discharge_hs,discharge_other,admission_source_er,admission_source_physician,admission_source_other,diag_other,diag_circulatory,diag_endocrine,diag_respiratory,diag_genitourinary
count,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,...,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000,98044.000000
mean,65.981804,4.403288,43.104657,1.336900,16.033230,0.373516,0.201960,0.646098,7.449053,0.461711,...,0.136959,0.138499,0.573059,0.136939,0.290002,0.813074,0.576833,0.222074,0.134562,0.171362
std,15.925716,2.986385,19.687546,1.703316,8.121909,1.279804,0.944477,1.273604,1.921457,0.498534,...,0.343806,0.345425,0.494636,0.343784,0.453765,0.389854,0.494064,0.415643,0.341257,0.376827
min,5.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,55.000000,2.000000,32.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,65.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,75.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,95.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 pierwsza kolumno to stary index niechce jej

In [5]:
df.drop(df.columns[0], axis = 1, inplace = True)

### zmapowanie wektorowych wartości na 0 i 1

In [7]:
from collections import defaultdict
d = defaultdict(lambda :1)
d['[0 0 1]'] = 0
df['readmitted']=df['readmitted'].map(d)

## podział na zbiory treningowe i testowe

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    df.drop('readmitted',axis = 1), # X
    df['readmitted'], # y
    test_size=0.3,
    stratify=df['readmitted'],
    random_state=37)

## wybór modelu
żeby mieć hiperparametry do zmiany i szybko liczący się model wybieram randomforestclassifier

## test czy działa zbiór

In [98]:
from sklearn.metrics import f1_score

In [9]:
rfc = RandomForestClassifier(random_state = 37, n_jobs = -1)
rfc.fit(X_train,Y_train)


RandomForestClassifier()

In [99]:
rfc_out = rfc.predict(X_test)
rfc_out_prob = rfc.predict_proba(X_test)
print(roc_auc_score(Y_test,rfc_out_prob[:,1]))
print(f1_score(Y_test,rfc_out))

0.6600572212993829
0.5473941368078176


## manualna optymalizacja hiperparametrów

In [100]:
rfc = RandomForestClassifier(
    n_estimators=210,
    criterion='entropy',
    max_depth=6,
    min_samples_split=0.1,
    min_samples_leaf=5,
    max_features=19,
    random_state = 37,
    n_jobs=-1,
    
)
rfc.fit(X_train,Y_train)
rfc_out = rfc.predict(X_test)
print(roc_auc_score(Y_test, rfc_out))
print(f1_score(Y_test, rfc_out))
print(f1_score(Y_train,rfc.predict(X_train)))

0.6208745637592553
0.5473941368078176
0.5501352648573175


# automatyczna optymalizacja

In [24]:
from scipy.stats import uniform
from scipy.stats import randint
random_state = 37

cross walidacja wykonana będzie na 5 zbiorach , optymalizacja będzie wg roc_auc 

## RandomizedSearchCV

In [93]:
from sklearn.model_selection import RandomizedSearchCV
rscv = RandomizedSearchCV(
                        RandomForestClassifier(random_state = random_state),
                        param_distributions={'n_estimators':randint(low = 0, high = 1500),
                                             'criterion':['gini','entropy'],
                                             'max_depth':randint(low = 1, high = 10),
                                             'min_samples_split':uniform(),
                                             'max_features':randint(low = 1, high = 76)},
                        n_jobs=3,
                        scoring='roc_auc',
                        refit= 'roc_auc',
                        n_iter=50,
                        random_state = random_state
)
rscv.fit(X_train,Y_train)


RandomizedSearchCV(estimator=RandomForestClassifier(random_state=37), n_iter=50,
                   n_jobs=3,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021025C56FA0>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000210267126D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021007B2A0A0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002100A115790>},
                   random_state=37, refit='roc_auc', scoring='roc_auc')

In [95]:
rscv.best_score_

0.6665895635922704

In [96]:
rscv.best_params_

{'criterion': 'gini',
 'max_depth': 6,
 'max_features': 22,
 'min_samples_split': 0.04412111139287256,
 'n_estimators': 83}

In [101]:
out  = rscv.predict(X_test)
print(roc_auc_score(Y_test,out))
print(f1_score(Y_test,out))

0.6224163579082086
0.5440039480177661


## GridSearchCV

In [37]:
from sklearn.model_selection import GridSearchCV
gscv = RandomizedSearchCV(
                        RandomForestClassifier(n_jobs = -1, random_state = random_state),
                        param_distributions={'n_estimators':randint(low = 0, high = 300),
                                             'criterion':['gini','entropy'],
                                             'max_depth':randint(low = 1, high = 10),
                                             'min_samples_split':uniform(),
                                             'max_features':randint(low = 1, high = 76)},
                        n_jobs=3,
                        scoring='roc_auc',
                        refit= 'roc_auc',
                        n_iter=50,
                        random_state = random_state)
gscv.fit(X_train,Y_train)


RandomizedSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=37),
                   n_jobs=3,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000021069B78D30>,
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002100E1C7BE0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002100E1C7CA0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002100E1C7430>},
                   random_state=37, refit='roc_auc', scoring='roc_auc')

In [38]:
gscv.best_score_

0.6554816296445758

In [39]:
gscv.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'max_features': 27,
 'min_samples_split': 0.19680428807852413,
 'n_estimators': 244}

In [103]:
out = gscv.predict(X_test)
print(roc_auc_score(Y_test,out))
print(f1_score(Y_test,out))

0.6060288010106185
0.5261565612000504


## Optymalizacja Bayesowska

In [74]:
import skopt,scipy
search_spaces = {
        'criterion': ['gini', 'entropy'],
        'n_estimators': skopt.space.space.Integer(1, 500),
        'max_depth': skopt.space.space.Integer(1, 20),
        'min_samples_split':skopt.space.space.Real(low = 0.00001,high=1),
        'max_features':skopt.space.space.Integer(1,76)
}

In [76]:
bo = skopt.BayesSearchCV(
        RandomForestClassifier(n_jobs = 1, random_state = random_state),
        search_spaces,
        n_jobs = 1,
        n_iter = 50,
        scoring = 'roc_auc',
        refit = 'roc_auc',
        random_state = random_state
)
bo.fit(X_train, Y_train)

BayesSearchCV(estimator=RandomForestClassifier(n_jobs=1, random_state=37),
              random_state=37, refit='roc_auc', scoring='roc_auc',
              search_spaces={'criterion': ['gini', 'entropy'],
                             'max_depth': Integer(low=1, high=20, prior='uniform', transform='normalize'),
                             'max_features': Integer(low=1, high=76, prior='uniform', transform='normalize'),
                             'min_samples_split': Real(low=1e-05, high=1, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=1, high=500, prior='uniform', transform='normalize')})

In [77]:
bo.best_score_

0.6845506161726673

In [78]:
bo.best_params_

OrderedDict([('criterion', 'entropy'),
             ('max_depth', 15),
             ('max_features', 10),
             ('min_samples_split', 1e-05),
             ('n_estimators', 186)])

In [102]:

print(roc_auc_score(Y_test,bo.predict_proba(X_test)[:,1]))
print(f1_score(Y_test, bo.predict(X_test)))

0.683000773865825
0.5551692740667163


## podsumowanie
najlepiej sprawdziła sie optymalizacja bayesowska, na drugim miejscu random a ostatni grid search, natomiast optymalizacja bayesowska liczyła się najdłużej.